# Homework 3

In this homework, you will test various time series classification methods. You must choose **three** datasets from [the UCR/UEA time series repository](http://timeseriesclassification.com) and perform the tasks by evaluating the models on three selected datasets.

Note that the questions are not 100% the same as the lab tasks. Please carefully read all descriptions. Compared to previous homework, there is no fixed answer, and we will evaluate your assignment based on your trials rather than the results. When the description does not specify or restrict things during your implementation process, you can choose your way freely. We will always grade based on the written description on each task only.


### Task 0: Preparation

You need to choose **three** datasets from the UCR/UEA time series repository. Please be careful since some UCR datasets can take a long time to be processed - You do not need to choose heavy datasets since it would slow your training/testing process. Use sktime's `load_UCR_UEA_dataset` function to perform. Please note that **you should use each dataset's original train/test splits** to train and report the test scores.

In [ ]:
from sktime.datasets import load_UCR_UEA_dataset
X_train_ecg200, y_train_ecg200 = load_UCR_UEA_dataset(name="ECG200", split="train", return_X_y=True, return_type="numpy3D")
X_test_ecg200, y_test_ecg200 = load_UCR_UEA_dataset(name="ECG200", split="test", return_X_y=True, return_type="numpy3D")

#coffee dataset
X_train_coffee, y_train_coffee = load_UCR_UEA_dataset(name="Coffee", split="train", return_X_y=True, return_type="numpy3D")
X_test_coffee, y_test_coffee = load_UCR_UEA_dataset(name="Coffee", split="test", return_X_y=True, return_type="numpy3D")


X_train_wafer, y_train_wafer = load_UCR_UEA_dataset("Wafer", split="train", return_X_y=True,return_type="numpy3D")
X_test_wafer, y_test_wafer = load_UCR_UEA_dataset("Wafer", split="test", return_X_y=True,return_type="numpy3D")

In [ ]:
!pip install sktime

### Task 1: Time series classification using deep learning 1



Time series classification problems can be solved using networks like CNN, RNN, or FCN. You can even try to merge different networks. In this task, you must test your three chosen datasets on four other models.

Try to implement four different models: 1) Fully connected network at least with five dense layers, 2) One directional RNN, 3) 1D-CNN only, and 4) 1D-CNN +
GRU. Note that you can always link the network to a fully connected layer to match the output size. You can freely construct any structure you want. Report the average test scores of four models on three datasets you chose. It would be four scores in total. Mark the best model in terms of the average test score. Briefly explain the structure you constructed. There is no definitive answer, and it is up to your own model. Here it would be best if you keep the following rules:

- When initially loading the dataset, use sktime's `load_UCR_UEA_dataset` function. This is for our grading purpose.
- You should use at least **two** Tensorflow callbacks when you fit your model. These can be built-in ones or your personalized callback.
- You should use Tensorflow's data API (`tf.data`) to manage your dataset and use `shuffle`, `batch,` and `prefetch` functions. This means that you need to convert the data format using the `from_tensor_slices` function. This also means that you need to create your own validation set. You are not limited to using any methods to do this, but you may also need to shuffle the dataset before (for that, you can use `np.random.permutation`). If you use Torch, explain how you implement the equivalent operations.
- You need to clearly report the **test accuracy** of the four models. Training and validation accuracy scores are not enough.
- You may need to deal with datasets of different sizes. In this case, it might be helpful to make a function to create a model that can receive different input sizes as a parameter.


In [ ]:
import tensorflow as tf
import numpy as np



X_train_ecg200, y_train_ecg200 = load_UCR_UEA_dataset(name="ECG200", split="train", return_X_y=True, return_type="numpy3D")
X_test_ecg200, y_test_ecg200 = load_UCR_UEA_dataset(name="ECG200", split="test", return_X_y=True, return_type="numpy3D")

if not isinstance(y_train_ecg200, np.ndarray):
    y_train_ecg200 = np.array(y_train_ecg200)
if not isinstance(y_test_ecg200, np.ndarray):
    y_test_ecg200 = np.array(y_test_ecg200)


y_train_ecg200 = y_train_ecg200.astype('int64')
y_test_ecg200 = y_test_ecg200.astype('int64')

def prepare_dataset(X, y, batch_size=32, shuffle_buffer_size=100):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle_buffer_size > 0:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

batch_size = 32


In [ ]:

print("Number of NaN values in X_train: ", np.isnan(X_train_ecg200).sum())
print("Number of NaN values in X_test: ", np.isnan(X_test_ecg200).sum())
print("Number of NaN values in y_train: ", np.isnan(y_train_ecg200).sum())
print("Number of NaN values in y_test: ", np.isnan(y_test_ecg200).sum())

Number of NaN values in X_train:  0
Number of NaN values in X_test:  0
Number of NaN values in y_train:  0
Number of NaN values in y_test:  0


In [ ]:

mean = X_train_ecg200.mean()
std = X_train_ecg200.std()

X_train_normalized = (X_train_ecg200 - mean) / std
X_test_normalized = (X_test_ecg200 - mean) / std


In [ ]:
X_train_normalized.shape, X_test_normalized.shape

((100, 1, 96), (100, 1, 96))

In [ ]:
import numpy as np
np.unique(y_train_ecg200)

array([-1,  1])

In [ ]:

y_train_ecg200[y_train_ecg200 == -1] = 0
y_test_ecg200[y_test_ecg200 == -1] = 0

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Conv1D, GRU, Flatten, Input
from tensorflow.keras.layers import MaxPooling1D
# 1. Fully Connected Network
def create_fcn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# 2. One Directional RNN
def create_rnn_model(input_shape, num_classes):
    model = Sequential([
        SimpleRNN(50, input_shape=input_shape, return_sequences=True),
        SimpleRNN(50),
        Dense(num_classes, activation='softmax')
    ])
    return model

# 3. 1D-CNN
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters=16, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# 4. 1D-CNN + GRU
def create_cnn_gru_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        GRU(50),
        Dense(num_classes, activation='softmax')
    ])
    return model


In [ ]:
# Reshape the data to (samples, timesteps, features)
X_train_ecg200 = X_train_normalized.reshape((-1, 96, 1))
X_test_ecg200 = X_test_normalized.reshape((-1, 96, 1))

train_dataset_ecg200 = prepare_dataset(X_train_ecg200, y_train_ecg200, batch_size=batch_size)
test_dataset_ecg200 = prepare_dataset(X_test_ecg200, y_test_ecg200, batch_size=batch_size)

input_shape_3d = (96, 1)  # 96 timesteps, 1 feature

num_classes = len(np.unique(y_train_ecg200))
fcn_model = create_fcn_model(input_shape_3d, num_classes)
rnn_model = create_rnn_model(input_shape_3d, num_classes)
cnn_model = create_cnn_model(input_shape_3d, num_classes)
cnn_gru_model = create_cnn_gru_model(input_shape_3d, num_classes)

In [ ]:
from tensorflow.keras.optimizers import Adam
def train_model(model, train_dataset, test_dataset, epochs=10, learning_rate=1e-5):
    # Check for NaNs just before training
    def check_nan(dataset):
        for features, labels in dataset:
            if tf.reduce_any(tf.math.is_nan(features)):
                raise ValueError("NaN values detected in dataset.")

    check_nan(train_dataset)
    check_nan(test_dataset)

    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
    ]

    optimizer = Adam(learning_rate=learning_rate, clipvalue=0.5)

    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=test_dataset,
        callbacks=callbacks
    )

    # Load the best weights, if available
    try:
        model.load_weights('best_model.h5')
    except FileNotFoundError:
        print("Checkpoint file not found. Using model as is after training.")

    test_loss, test_accuracy = model.evaluate(test_dataset)
    return history, test_loss, test_accuracy

learning_rate=1e-5

In [ ]:
# Train and evaluate each model
fcn_history, fcn_test_loss, fcn_accuracy = train_model(fcn_model, train_dataset_ecg200, test_dataset_ecg200, epochs=10, learning_rate=learning_rate)

Epoch 1/10
1/4 [======>.......................] - ETA: 4s - loss: 0.6840 - accuracy: 0.5312
Epoch 1: val_loss improved from inf to 0.69020, saving model to best_model.h5
4/4 [==============================] - 2s 86ms/step - loss: 0.6926 - accuracy: 0.5200 - val_loss: 0.6902 - val_accuracy: 0.5800
Epoch 2/10
1/4 [======>.......................] - ETA: 0s - loss: 0.6838 - accuracy: 0.5625
Epoch 2: val_loss improved from 0.69020 to 0.68377, saving model to best_model.h5
4/4 [==============================] - 0s 27ms/step - loss: 0.6837 - accuracy: 0.6600 - val_loss: 0.6838 - val_accuracy: 0.6400


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 3/10
1/4 [======>.......................] - ETA: 0s - loss: 0.6748 - accuracy: 0.7500
Epoch 3: val_loss improved from 0.68377 to 0.67770, saving model to best_model.h5
4/4 [==============================] - 0s 33ms/step - loss: 0.6766 - accuracy: 0.7200 - val_loss: 0.6777 - val_accuracy: 0.6500
Epoch 4/10
1/4 [======>.......................] - ETA: 0s - loss: 0.6695 - accuracy: 0.8125
Epoch 4: val_loss improved from 0.67770 to 0.67233, saving model to best_model.h5
4/4 [==============================] - 0s 27ms/step - loss: 0.6697 - accuracy: 0.7200 - val_loss: 0.6723 - val_accuracy: 0.6500
Epoch 5/10
1/4 [======>.......................] - ETA: 0s - loss: 0.6830 - accuracy: 0.6875
Epoch 5: val_loss improved from 0.67233 to 0.66769, saving model to best_model.h5
4/4 [==============================] - 0s 35ms/step - loss: 0.6632 - accuracy: 0.7200 - val_loss: 0.6677 - val_accuracy: 0.6500
Epoch 6/10
1/4 [======>.......................] - ETA: 0s - loss: 0.6625 - accuracy: 0.6875
Ep

In [ ]:
rnn_history, rnn_test_loss, rnn_accuracy = train_model(rnn_model, train_dataset_ecg200, test_dataset_ecg200, epochs=10, learning_rate=learning_rate)


Epoch 1/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6878 - accuracy: 0.5729
Epoch 1: val_loss improved from inf to 0.67654, saving model to best_model.h5
4/4 [==============================] - 3s 185ms/step - loss: 0.6852 - accuracy: 0.5800 - val_loss: 0.6765 - val_accuracy: 0.6300
Epoch 2/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6791 - accuracy: 0.6250
Epoch 2: val_loss improved from 0.67654 to 0.67170, saving model to best_model.h5
4/4 [==============================] - 0s 75ms/step - loss: 0.6780 - accuracy: 0.6200 - val_loss: 0.6717 - val_accuracy: 0.6500
Epoch 3/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6745 - accuracy: 0.6354
Epoch 3: val_loss improved from 0.67170 to 0.66705, saving model to best_model.h5
4/4 [==============================] - 0s 72ms/step - loss: 0.6712 - accuracy: 0.6400 - val_loss: 0.6671 - val_accuracy: 0.6800
Epoch 4/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6660 - accuracy: 0.6354
Epoch

In [ ]:
try:
    cnn_history, cnn_test_loss, cnn_accuracy = train_model(cnn_model, train_dataset_ecg200, test_dataset_ecg200, epochs=10, learning_rate=learning_rate)
except Exception as e:
    print("An error occurred during training:", e)

Epoch 1/10
1/4 [======>.......................] - ETA: 2s - loss: 0.7003 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 0.65791, saving model to best_model.h5
4/4 [==============================] - 1s 67ms/step - loss: 0.6746 - accuracy: 0.5600 - val_loss: 0.6579 - val_accuracy: 0.6100
Epoch 2/10
1/4 [======>.......................] - ETA: 0s - loss: 0.6749 - accuracy: 0.6250
Epoch 2: val_loss improved from 0.65791 to 0.65411, saving model to best_model.h5
4/4 [==============================] - 0s 19ms/step - loss: 0.6697 - accuracy: 0.5600 - val_loss: 0.6541 - val_accuracy: 0.6100
Epoch 3/10
1/4 [======>.......................] - ETA: 0s - loss: 0.6804 - accuracy: 0.5625
Epoch 3: val_loss improved from 0.65411 to 0.65051, saving model to best_model.h5
4/4 [==============================] - 0s 17ms/step - loss: 0.6648 - accuracy: 0.5700 - val_loss: 0.6505 - val_accuracy: 0.6300
Epoch 4/10
1/4 [======>.......................] - ETA: 0s - loss: 0.6411 - accuracy: 0.5625
Epoch 

In [ ]:
cnn_gru_history, cnn_gru_test_loss, cnn_gru_accuracy = train_model(cnn_gru_model, train_dataset_ecg200, test_dataset_ecg200, epochs=10, learning_rate=learning_rate)

Epoch 1/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6964 - accuracy: 0.4062
Epoch 1: val_loss improved from inf to 0.69588, saving model to best_model.h5
4/4 [==============================] - 4s 329ms/step - loss: 0.6964 - accuracy: 0.4000 - val_loss: 0.6959 - val_accuracy: 0.4300
Epoch 2/10
4/4 [==============================] - ETA: 0s - loss: 0.6960 - accuracy: 0.4300
Epoch 2: val_loss improved from 0.69588 to 0.69561, saving model to best_model.h5
4/4 [==============================] - 0s 122ms/step - loss: 0.6960 - accuracy: 0.4300 - val_loss: 0.6956 - val_accuracy: 0.4500
Epoch 3/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6956 - accuracy: 0.4375
Epoch 3: val_loss improved from 0.69561 to 0.69535, saving model to best_model.h5
4/4 [==============================] - 0s 86ms/step - loss: 0.6956 - accuracy: 0.4400 - val_loss: 0.6954 - val_accuracy: 0.4500
Epoch 4/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6951 - accuracy: 0.4688
Epoc

In [ ]:
fcn_test_loss, fcn_test_accuracy = fcn_model.evaluate(X_test_ecg200, y_test_ecg200, verbose=0)
print(f"Fully Connected Network Test Accuracy: {fcn_test_accuracy:.4f}")

rnn_test_loss, rnn_test_accuracy = rnn_model.evaluate(X_test_ecg200, y_test_ecg200, verbose=0)
print(f"One Directional RNN Test Accuracy: {rnn_test_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(X_test_ecg200, y_test_ecg200, verbose=0)
print(f"1D-CNN Test Accuracy: {cnn_test_accuracy:.4f}")

cnn_gru_test_loss, cnn_gru_test_accuracy = cnn_gru_model.evaluate(X_test_ecg200, y_test_ecg200, verbose=0)
print(f"1D-CNN + GRU Test Accuracy: {cnn_gru_test_accuracy:.4f}")

Fully Connected Network Test Accuracy: 0.6800
One Directional RNN Test Accuracy: 0.7700
1D-CNN Test Accuracy: 0.6600
1D-CNN + GRU Test Accuracy: 0.5900


In [ ]:
#coffee dataset
X_train_coffee, y_train_coffee = load_UCR_UEA_dataset(name="Coffee", split="train", return_X_y=True, return_type="numpy3D")
X_test_coffee, y_test_coffee = load_UCR_UEA_dataset(name="Coffee", split="test", return_X_y=True, return_type="numpy3D")

In [ ]:
if not isinstance(y_train_coffee, np.ndarray):
    y_train_coffee = np.array(y_train_coffee)
if not isinstance(y_test_coffee, np.ndarray):
    y_test_coffee = np.array(y_test_coffee)


y_train_coffee = y_train_coffee.astype('int64')
y_test_coffee = y_test_coffee.astype('int64')

In [ ]:

print("Number of NaN values in X_train: ", np.isnan(X_train_coffee).sum())
print("Number of NaN values in X_test: ", np.isnan(X_test_coffee).sum())
print("Number of NaN values in y_train: ", np.isnan(y_train_coffee).sum())
print("Number of NaN values in y_test: ", np.isnan(y_test_coffee).sum())

Number of NaN values in X_train:  0
Number of NaN values in X_test:  0
Number of NaN values in y_train:  0
Number of NaN values in y_test:  0


In [ ]:
mean = X_train_coffee.mean()
std = X_train_coffee.std()

X_train_normalized = (X_train_coffee- mean) / std
X_test_normalized = (X_test_coffee - mean) / std


In [ ]:
X_train_normalized.shape, X_test_normalized.shape

((28, 1, 286), (28, 1, 286))

In [ ]:
import numpy as np
np.unique(y_train_coffee)

array([0, 1])

In [ ]:
# Reshape the data to (samples, timesteps, features)
X_train_coffee= X_train_normalized.reshape((-1, 286, 1))
X_test_coffee = X_test_normalized.reshape((-1,286, 1))

train_dataset_coffee = prepare_dataset(X_train_coffee, y_train_coffee, batch_size=batch_size)
test_dataset_coffee = prepare_dataset(X_test_coffee, y_test_coffee, batch_size=batch_size)

input_shape_3d = (286, 1)  # 286 timesteps, 1 feature

num_classes = len(np.unique(y_train_coffee))
fcn_model = create_fcn_model(input_shape_3d, num_classes)
rnn_model = create_rnn_model(input_shape_3d, num_classes)
cnn_model = create_cnn_model(input_shape_3d, num_classes)
cnn_gru_model = create_cnn_gru_model(input_shape_3d, num_classes)

In [ ]:
# Train and evaluate each model
fcn_history, fcn_test_loss, fcn_accuracy = train_model(fcn_model, train_dataset_coffee,test_dataset_coffee, epochs=10, learning_rate=learning_rate)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.6907 - accuracy: 0.5357
Epoch 1: val_loss improved from inf to 0.69170, saving model to best_model.h5
1/1 [==============================] - 2s 2s/step - loss: 0.6907 - accuracy: 0.5357 - val_loss: 0.6917 - val_accuracy: 0.4643
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.6868 - accuracy: 0.5357
Epoch 2: val_loss improved from 0.69170 to 0.68892, saving model to best_model.h5
1/1 [==============================] - 0s 108ms/step - loss: 0.6868 - accuracy: 0.5357 - val_loss: 0.6889 - val_accuracy: 0.4643


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.6833 - accuracy: 0.5714
Epoch 3: val_loss improved from 0.68892 to 0.68637, saving model to best_model.h5
1/1 [==============================] - 0s 102ms/step - loss: 0.6833 - accuracy: 0.5714 - val_loss: 0.6864 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.6800 - accuracy: 0.5714
Epoch 4: val_loss improved from 0.68637 to 0.68410, saving model to best_model.h5
1/1 [==============================] - 0s 106ms/step - loss: 0.6800 - accuracy: 0.5714 - val_loss: 0.6841 - val_accuracy: 0.5357
Epoch 5/10
1/1 [==============================] - ETA: 0s - loss: 0.6770 - accuracy: 0.5714
Epoch 5: val_loss improved from 0.68410 to 0.68206, saving model to best_model.h5
1/1 [==============================] - 0s 105ms/step - loss: 0.6770 - accuracy: 0.5714 - val_loss: 0.6821 - val_accuracy: 0.5357
Epoch 6/10
1/1 [==============================] - ETA: 0s - loss: 0.6743 - accuracy: 0.6429

In [ ]:
rnn_history, rnn_test_loss, rnn_accuracy = train_model(rnn_model, train_dataset_coffee, test_dataset_coffee, epochs=10, learning_rate=learning_rate)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.7657 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 0.79108, saving model to best_model.h5
1/1 [==============================] - 3s 3s/step - loss: 0.7657 - accuracy: 0.5000 - val_loss: 0.7911 - val_accuracy: 0.4643
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.7645 - accuracy: 0.5000
Epoch 2: val_loss improved from 0.79108 to 0.78962, saving model to best_model.h5
1/1 [==============================] - 0s 241ms/step - loss: 0.7645 - accuracy: 0.5000 - val_loss: 0.7896 - val_accuracy: 0.4643
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.7633 - accuracy: 0.5000
Epoch 3: val_loss improved from 0.78962 to 0.78817, saving model to best_model.h5
1/1 [==============================] - 0s 244ms/step - loss: 0.7633 - accuracy: 0.5000 - val_loss: 0.7882 - val_accuracy: 0.4643
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.7620 - accuracy: 0.5000
Epoch 

In [ ]:
try:
    cnn_history, cnn_test_loss, cnn_accuracy = train_model(cnn_model, train_dataset_coffee, test_dataset_coffee, epochs=10, learning_rate=learning_rate)
except Exception as e:
    print("An error occurred during training:", e)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.7833 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 0.74963, saving model to best_model.h5
1/1 [==============================] - 1s 1s/step - loss: 0.7833 - accuracy: 0.5000 - val_loss: 0.7496 - val_accuracy: 0.5357
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.7806 - accuracy: 0.5000
Epoch 2: val_loss improved from 0.74963 to 0.74748, saving model to best_model.h5
1/1 [==============================] - 0s 60ms/step - loss: 0.7806 - accuracy: 0.5000 - val_loss: 0.7475 - val_accuracy: 0.5357
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.7779 - accuracy: 0.5000
Epoch 3: val_loss improved from 0.74748 to 0.74536, saving model to best_model.h5
1/1 [==============================] - 0s 57ms/step - loss: 0.7779 - accuracy: 0.5000 - val_loss: 0.7454 - val_accuracy: 0.5357
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.7752 - accuracy: 0.5000
Epoch 4:

In [ ]:
cnn_gru_history, cnn_gru_test_loss, cnn_gru_accuracy = train_model(cnn_gru_model, train_dataset_coffee, test_dataset_coffee, epochs=10, learning_rate=learning_rate)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.7162 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 0.72957, saving model to best_model.h5
1/1 [==============================] - 4s 4s/step - loss: 0.7162 - accuracy: 0.5000 - val_loss: 0.7296 - val_accuracy: 0.4643
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.7159 - accuracy: 0.5000
Epoch 2: val_loss improved from 0.72957 to 0.72910, saving model to best_model.h5
1/1 [==============================] - 0s 382ms/step - loss: 0.7159 - accuracy: 0.5000 - val_loss: 0.7291 - val_accuracy: 0.4643
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.7155 - accuracy: 0.5000
Epoch 3: val_loss improved from 0.72910 to 0.72863, saving model to best_model.h5
1/1 [==============================] - 0s 468ms/step - loss: 0.7155 - accuracy: 0.5000 - val_loss: 0.7286 - val_accuracy: 0.4643
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.7152 - accuracy: 0.5000
Epoch 

In [ ]:
fcn_test_loss, fcn_test_accuracy = fcn_model.evaluate(X_test_coffee, y_test_coffee, verbose=0)
print(f"Fully Connected Network Test Accuracy: {fcn_test_accuracy:.4f}")

rnn_test_loss, rnn_test_accuracy = rnn_model.evaluate(X_test_coffee, y_test_coffee, verbose=0)
print(f"One Directional RNN Test Accuracy: {rnn_test_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(X_test_coffee, y_test_coffee, verbose=0)
print(f"1D-CNN Test Accuracy: {cnn_test_accuracy:.4f}")

cnn_gru_test_loss, cnn_gru_test_accuracy = cnn_gru_model.evaluate(X_test_coffee, y_test_coffee, verbose=0)
print(f"1D-CNN + GRU Test Accuracy: {cnn_gru_test_accuracy:.4f}")

Fully Connected Network Test Accuracy: 0.6071
One Directional RNN Test Accuracy: 0.4643
1D-CNN Test Accuracy: 0.5357
1D-CNN + GRU Test Accuracy: 0.4643


In [ ]:

X_train_wafer, y_train_wafer = load_UCR_UEA_dataset("Wafer", split="train", return_X_y=True,return_type="numpy3D")
X_test_wafer, y_test_wafer = load_UCR_UEA_dataset("Wafer", split="test", return_X_y=True,return_type="numpy3D")


In [ ]:
if not isinstance(y_train_wafer, np.ndarray):
    y_train_wafer = np.array(y_train_wafer)
if not isinstance(y_test_wafer, np.ndarray):
    y_test_itay_train_wafer = np.array(y_test_wafer)

y_train_wafer = y_train_wafer.astype('int64')
y_test_wafer = y_test_wafer.astype('int64')

In [ ]:
print("Number of NaN values in X_train: ", np.isnan(X_train_wafer).sum())
print("Number of NaN values in X_test: ", np.isnan(X_test_wafer).sum())
print("Number of NaN values in y_train: ", np.isnan(y_train_wafer).sum())
print("Number of NaN values in y_test: ", np.isnan(y_test_wafer).sum())

Number of NaN values in X_train:  0
Number of NaN values in X_test:  0
Number of NaN values in y_train:  0
Number of NaN values in y_test:  0


In [ ]:
mean = X_train_wafer.mean()
std = X_train_wafer.std()

X_train_normalized = (X_train_wafer- mean) / std
X_test_normalized = (X_test_wafer - mean) / std

In [ ]:
X_train_normalized.shape, X_test_normalized.shape

((1000, 1, 152), (6164, 1, 152))

In [ ]:
import numpy as np
np.unique(y_train_wafer)

array([-1,  1])

In [ ]:
y_train_wafer[y_train_wafer == -1] = 0
y_test_wafer[y_test_wafer == -1] = 0

In [ ]:
# Reshape the data to (samples, timesteps, features)
X_train_wafer= X_train_normalized.reshape((-1, 152, 1))
X_test_wafer = X_test_normalized.reshape((-1,152, 1))

train_dataset_wafer = prepare_dataset(X_train_wafer, y_train_wafer, batch_size=batch_size)
test_dataset_wafer = prepare_dataset(X_test_wafer, y_test_wafer, batch_size=batch_size)

input_shape_3d = (152, 1)  # 286 timesteps, 1 feature

num_classes = len(np.unique(y_train_wafer))
fcn_model = create_fcn_model(input_shape_3d, num_classes)
rnn_model = create_rnn_model(input_shape_3d, num_classes)
cnn_model = create_cnn_model(input_shape_3d, num_classes)
cnn_gru_model = create_cnn_gru_model(input_shape_3d, num_classes)

In [ ]:
# Train and evaluate each model
fcn_history, fcn_test_loss, fcn_accuracy = train_model(fcn_model, train_dataset_wafer, test_dataset_wafer, epochs=10, learning_rate=learning_rate)

Epoch 1/10
30/32 [===========================>..] - ETA: 0s - loss: 0.7309 - accuracy: 0.3938
Epoch 1: val_loss improved from inf to 0.66632, saving model to best_model.h5
32/32 [==============================] - 2s 29ms/step - loss: 0.7285 - accuracy: 0.4050 - val_loss: 0.6663 - val_accuracy: 0.6650


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.6244 - accuracy: 0.7232
Epoch 2: val_loss improved from 0.66632 to 0.58693, saving model to best_model.h5
32/32 [==============================] - 1s 30ms/step - loss: 0.6208 - accuracy: 0.7320 - val_loss: 0.5869 - val_accuracy: 0.7967
Epoch 3/10
29/32 [==========================>...] - ETA: 0s - loss: 0.5489 - accuracy: 0.8869
Epoch 3: val_loss improved from 0.58693 to 0.51642, saving model to best_model.h5
32/32 [==============================] - 1s 34ms/step - loss: 0.5473 - accuracy: 0.8850 - val_loss: 0.5164 - val_accuracy: 0.9031
Epoch 4/10
28/32 [=========================>....] - ETA: 0s - loss: 0.4690 - accuracy: 0.9219
Epoch 4: val_loss improved from 0.51642 to 0.43556, saving model to best_model.h5
32/32 [==============================] - 3s 96ms/step - loss: 0.4654 - accuracy: 0.9200 - val_loss: 0.4356 - val_accuracy: 0.9114
Epoch 5/10
29/32 [==========================>...] - ETA: 0s - loss: 0.3928 - accur

In [ ]:
rnn_history, rnn_test_loss, rnn_accuracy = train_model(rnn_model, train_dataset_wafer, test_dataset_wafer, epochs=10, learning_rate=learning_rate)

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 0.6486 - accuracy: 0.6810
Epoch 1: val_loss improved from inf to 0.65152, saving model to best_model.h5
32/32 [==============================] - 10s 196ms/step - loss: 0.6486 - accuracy: 0.6810 - val_loss: 0.6515 - val_accuracy: 0.6535
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.6326 - accuracy: 0.6790
Epoch 2: val_loss improved from 0.65152 to 0.64249, saving model to best_model.h5
32/32 [==============================] - 13s 422ms/step - loss: 0.6326 - accuracy: 0.6790 - val_loss: 0.6425 - val_accuracy: 0.6549
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.6222 - accuracy: 0.6820
Epoch 3: val_loss improved from 0.64249 to 0.63621, saving model to best_model.h5
32/32 [==============================] - 8s 262ms/step - loss: 0.6222 - accuracy: 0.6820 - val_loss: 0.6362 - val_accuracy: 0.6570
Epoch 4/10
32/32 [==============================] - ETA: 0s - loss: 0.6145 - accu

In [ ]:
try:
    cnn_history, cnn_test_loss, cnn_accuracy = train_model(cnn_model, train_dataset_wafer, test_dataset_wafer, epochs=10, learning_rate=learning_rate)
except Exception as e:
    print("An error occurred during training:", e)

Epoch 1/10
22/32 [===================>..........] - ETA: 0s - loss: 0.7477 - accuracy: 0.3366
Epoch 1: val_loss improved from inf to 0.66890, saving model to best_model.h5
32/32 [==============================] - 2s 31ms/step - loss: 0.7308 - accuracy: 0.4040 - val_loss: 0.6689 - val_accuracy: 0.6531
Epoch 2/10
26/32 [=======================>......] - ETA: 0s - loss: 0.6398 - accuracy: 0.7572
Epoch 2: val_loss improved from 0.66890 to 0.58751, saving model to best_model.h5
32/32 [==============================] - 1s 25ms/step - loss: 0.6351 - accuracy: 0.7670 - val_loss: 0.5875 - val_accuracy: 0.8650
Epoch 3/10
23/32 [====================>.........] - ETA: 0s - loss: 0.5676 - accuracy: 0.8777
Epoch 3: val_loss improved from 0.58751 to 0.52310, saving model to best_model.h5
32/32 [==============================] - 1s 26ms/step - loss: 0.5591 - accuracy: 0.8760 - val_loss: 0.5231 - val_accuracy: 0.8787
Epoch 4/10
31/32 [============================>.] - ETA: 0s - loss: 0.4994 - accuracy:

In [ ]:
cnn_gru_history, cnn_gru_test_loss, cnn_gru_accuracy = train_model(cnn_gru_model, train_dataset_wafer, test_dataset_wafer, epochs=10, learning_rate=learning_rate)

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 0.6859 - accuracy: 0.6620
Epoch 1: val_loss improved from inf to 0.67667, saving model to best_model.h5
32/32 [==============================] - 11s 273ms/step - loss: 0.6859 - accuracy: 0.6620 - val_loss: 0.6767 - val_accuracy: 0.6639
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.6661 - accuracy: 0.8420
Epoch 2: val_loss improved from 0.67667 to 0.65786, saving model to best_model.h5
32/32 [==============================] - 13s 425ms/step - loss: 0.6661 - accuracy: 0.8420 - val_loss: 0.6579 - val_accuracy: 0.8921
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.6476 - accuracy: 0.9030
Epoch 3: val_loss improved from 0.65786 to 0.64057, saving model to best_model.h5
32/32 [==============================] - 13s 421ms/step - loss: 0.6476 - accuracy: 0.9030 - val_loss: 0.6406 - val_accuracy: 0.8921
Epoch 4/10
32/32 [==============================] - ETA: 0s - loss: 0.6302 - acc

In [ ]:
fcn_test_loss, fcn_test_accuracy = fcn_model.evaluate(X_test_wafer, y_test_wafer, verbose=0)
print(f"Fully Connected Network Test Accuracy: {fcn_test_accuracy:.4f}")

rnn_test_loss, rnn_test_accuracy = rnn_model.evaluate(X_test_wafer, y_test_wafer, verbose=0)
print(f"One Directional RNN Test Accuracy: {rnn_test_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(X_test_wafer, y_test_wafer, verbose=0)
print(f"1D-CNN Test Accuracy: {cnn_test_accuracy:.4f}")

cnn_gru_test_loss, cnn_gru_test_accuracy = cnn_gru_model.evaluate(X_test_wafer, y_test_wafer, verbose=0)
print(f"1D-CNN + GRU Test Accuracy: {cnn_gru_test_accuracy:.4f}")

Fully Connected Network Test Accuracy: 0.9486
One Directional RNN Test Accuracy: 0.6799
1D-CNN Test Accuracy: 0.8913
1D-CNN + GRU Test Accuracy: 0.8921


### Task 2: Time series classification using deep learning 2

There has been several neural network models dedicated to time series classification. Besides your own models that you developed in Lab 4, now you will develop such dedicated models by referring to some papers, and test if they indeed perform better than your rough models. There are two famous papers as follows:
 - [Convolutional neural networks for time series classification (2017)](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7870510)
 - [Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline (2017)](https://arxiv.org/abs/1611.06455)

First paper's idea is already implemented in sktime, with the name `CNNClassifier`. Second paper has three models and those are easy to develop using tensorflow. Now the task is to develop two models (MLP and FCN) in the second paper and test it together with `CNNClassifier`.

Use the same four datasets, and test sktime's `CNNClassifier` and MLP and FCN models you develop. Report test scores of three models (`CNNClassifier`, MLP, and FCN) on three datasets you chose. It would be nine scores in total. For MLP and FCN, you may need to satisfy the following requirement:

- You should use at least **two** Tensorflow callbacks when you fit your model. These can be built-in ones or your personalized callback. If you use Torch, explain how you implement the equivalent operations.
- You should run the model at least 10 epochs.
- You can use the same processed datasets in Task 1. For `CNNClassifier`, as you cannot use `tf.Data`, you may put the training set directly.
- For `CNNClassifier`, you can run it with the default parameters or reduce the number of epoch (default is 2000).
- Please use the predefined test dataset to report the test scores.

Note that the main purpose of this task is to check if you can develop a similar network structure with description. If the detail of the specific part (e.g., size of one layer or some custom parameters like epoch) is missing in the paper, you can set it on your own.


In [ ]:
from sktime.classification.deep_learning.cnn import CNNClassifier
from sktime.datasets import load_unit_test
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Conv1D, GRU, Flatten, Input,Dropout
from tensorflow.keras.layers import MaxPooling1D


In [ ]:
#ecg200
X_train_ecg200, y_train_ecg200 = load_UCR_UEA_dataset(name="ECG200", split="train", return_X_y=True, return_type="numpy3D")
X_test_ecg200, y_test_ecg200= load_UCR_UEA_dataset(name="ECG200", split="test", return_X_y=True, return_type="numpy3D")

cnn = CNNClassifier(n_epochs=200)
cnn.fit(X_train_ecg200, y_train_ecg200)
cnn_score = cnn.score(X_test_ecg200,y_test_ecg200)
print("CNNClassifier Test Accuracy:", cnn_score)

7/7 [==============================] - 0s 3ms/step
CNNClassifier Test Accuracy: 0.82


In [ ]:
#coffee
X_train_coffee, y_train_coffee = load_UCR_UEA_dataset(name="Coffee", split="train", return_X_y=True, return_type="numpy3D")
X_test_coffee, y_test_coffee= load_UCR_UEA_dataset(name="Coffee", split="test", return_X_y=True, return_type="numpy3D")

cnn = CNNClassifier(n_epochs=200)
cnn.fit(X_train_coffee, y_train_coffee)
cnn_score = cnn.score(X_test_coffee,y_test_coffee)
print("CNNClassifier Test Accuracy:", cnn_score)

2/2 [==============================] - 0s 8ms/step
CNNClassifier Test Accuracy: 1.0


In [ ]:
#wafer
X_train_wafer, y_train_wafer = load_UCR_UEA_dataset(name="Wafer", split="train", return_X_y=True, return_type="numpy3D")
X_test_wafer, y_test_wafer= load_UCR_UEA_dataset(name="Wafer", split="test", return_X_y=True, return_type="numpy3D")

cnn = CNNClassifier(n_epochs=200)
cnn.fit(X_train_wafer, y_train_wafer)
cnn_score = cnn.score(X_test_wafer,y_test_wafer)
print("CNNClassifier Test Accuracy:", cnn_score)

386/386 [==============================] - 2s 4ms/step
CNNClassifier Test Accuracy: 0.9928617780661908


In [ ]:
def create_mlp_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
def create_fcn_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Conv1D(filters=128, kernel_size=8, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def train_and_evaluate_model(model, train_dataset, test_dataset, epochs=10, learning_rate=1e-5):
    def check_for_nans(dataset):
        for features, labels in dataset:
            if tf.reduce_any(tf.math.is_nan(features)) or tf.reduce_any(tf.math.is_nan(labels)):
                raise ValueError("NaN values detected in the dataset.")

    check_for_nans(train_dataset)
    check_for_nans(test_dataset)

    optimizer = Adam(learning_rate=learning_rate, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, verbose=1),
        ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
    ]
    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=test_dataset,
        callbacks=callbacks
    )
    try:
        model.load_weights('best_model.h5')
    except FileNotFoundError as e:
        print(f"Checkpoint file not found: {e}. Using model as is after training.")

    test_loss, test_accuracy = model.evaluate(test_dataset)
    return history, test_loss, test_accuracy

In [ ]:
def prepare_dataset(X, y, batch_size=32, shuffle_buffer_size=100):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle_buffer_size > 0:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
#ecg200 dataset

X_train_ecg200, y_train_ecg200 = load_UCR_UEA_dataset(name="ECG200", split="train", return_X_y=True, return_type="numpy3D")
X_test_ecg200, y_test_ecg200 = load_UCR_UEA_dataset(name="ECG200", split="test", return_X_y=True, return_type="numpy3D")

if not isinstance(y_train_ecg200, np.ndarray):
    y_train_ecg200 = np.array(y_train_ecg200)
if not isinstance(y_test_ecg200, np.ndarray):
    y_test_ecg200 = np.array(y_test_ecg200)

y_train_ecg200 = y_train_ecg200.astype('int64')
y_test_ecg200 = y_test_ecg200.astype('int64')

mean = X_train_ecg200.mean()
std = X_train_ecg200.std()

X_train_ecg200 = (X_train_ecg200 - mean) / std
X_test_ecg200 = (X_test_ecg200 - mean) / std

y_train_ecg200[y_train_ecg200 == -1] = 0
y_test_ecg200[y_test_ecg200 == -1] = 0

X_train_ecg200 = X_train_ecg200.reshape((-1, 96, 1))
X_test_ecg200 = X_test_ecg200.reshape((-1, 96, 1))

train_dataset_ecg200 = prepare_dataset(X_train_ecg200, y_train_ecg200, batch_size=batch_size)
test_dataset_ecg200 = prepare_dataset(X_test_ecg200, y_test_ecg200, batch_size=batch_size)
num_classes = len(np.unique(y_train_ecg200))

input_shape_3d = (96, 1)
mlp_model = create_mlp_model(input_shape_3d, num_classes)
fcn_model = create_fcn_model(input_shape_3d, num_classes)

In [ ]:
mlp_history, mlp_test_loss, mlp_accuracy = train_model(mlp_model, train_dataset_ecg200, test_dataset_ecg200, epochs=10, learning_rate=learning_rate)

Epoch 1/10
1/4 [======>.......................] - ETA: 3s - loss: 0.7929 - accuracy: 0.3125
Epoch 1: val_loss improved from inf to 0.74581, saving model to best_model.h5
4/4 [==============================] - 1s 85ms/step - loss: 0.7478 - accuracy: 0.4300 - val_loss: 0.7458 - val_accuracy: 0.2200
Epoch 2/10
1/4 [======>.......................] - ETA: 0s - loss: 0.7407 - accuracy: 0.4375
Epoch 2: val_loss improved from 0.74581 to 0.72618, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 37ms/step - loss: 0.7360 - accuracy: 0.4700 - val_loss: 0.7262 - val_accuracy: 0.2100
Epoch 3/10
1/4 [======>.......................] - ETA: 0s - loss: 0.7015 - accuracy: 0.5000
Epoch 3: val_loss improved from 0.72618 to 0.70889, saving model to best_model.h5
4/4 [==============================] - 0s 33ms/step - loss: 0.7070 - accuracy: 0.5000 - val_loss: 0.7089 - val_accuracy: 0.3700
Epoch 4/10
1/4 [======>.......................] - ETA: 0s - loss: 0.6840 - accuracy: 0.5312
Epoch 4: val_loss improved from 0.70889 to 0.69407, saving model to best_model.h5
4/4 [==============================] - 0s 34ms/step - loss: 0.7024 - accuracy: 0.4800 - val_loss: 0.6941 - val_accuracy: 0.5700
Epoch 5/10
1/4 [======>.......................] - ETA: 0s - loss: 0.7222 - accuracy: 0.4688
Epoch 5: val_loss improved from 0.69407 to 0.68022, saving model to best_model.h5
4/4 [==============================] - 0s 33ms/step - loss: 0.6770 - accuracy: 0.5400 - val_lo

In [ ]:
fcn_history, fcn_test_loss, fcn_accuracy = train_model(fcn_model, train_dataset_ecg200, test_dataset_ecg200, epochs=10, learning_rate=learning_rate)

Epoch 1/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6750 - accuracy: 0.6354
Epoch 1: val_loss improved from inf to 0.68706, saving model to best_model.h5
4/4 [==============================] - 4s 365ms/step - loss: 0.6759 - accuracy: 0.6400 - val_loss: 0.6871 - val_accuracy: 0.6400
Epoch 2/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6513 - accuracy: 0.7708
Epoch 2: val_loss did not improve from 0.68706
4/4 [==============================] - 1s 154ms/step - loss: 0.6556 - accuracy: 0.7500 - val_loss: 0.6901 - val_accuracy: 0.6400
Epoch 3/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6457 - accuracy: 0.7396
Epoch 3: val_loss did not improve from 0.68706
4/4 [==============================] - 1s 152ms/step - loss: 0.6449 - accuracy: 0.7500 - val_loss: 0.6929 - val_accuracy: 0.4500
Epoch 4/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6271 - accuracy: 0.7708
Epoch 4: val_loss did not improve from 0.68706
4/4 [=====================

In [ ]:
fcn_test_loss, fcn_test_accuracy = fcn_model.evaluate(X_test_ecg200, y_test_ecg200, verbose=0)
print(f"Fully Connected Network Test Accuracy: {fcn_test_accuracy:.4f}")

mlp_test_loss, mlp_test_accuracy = mlp_model.evaluate(X_test_ecg200, y_test_ecg200, verbose=0)
print(f"MLP Test Accuracy: {mlp_test_accuracy:.4f}")

Fully Connected Network Test Accuracy: 0.6400
MLP Test Accuracy: 0.6600


In [ ]:
#Coffee dataset
#num_classes = len(np.unique(y_train_coffee))
X_train_coffee, y_train_coffee = load_UCR_UEA_dataset(name="Coffee", split="train", return_X_y=True, return_type="numpy3D")
X_test_coffee, y_test_coffee = load_UCR_UEA_dataset(name="Coffee", split="test", return_X_y=True, return_type="numpy3D")

if not isinstance(y_train_coffee, np.ndarray):
    y_train_coffee = np.array(y_train_coffee)
if not isinstance(y_test_coffee, np.ndarray):
    y_test_coffee = np.array(y_test_coffee)

y_train_coffee = y_train_coffee.astype('int64')
y_test_coffee = y_test_coffee.astype('int64')

mean = X_train_coffee.mean()
std = X_train_coffee.std()

X_train_coffee = (X_train_coffee - mean) / std
X_test_coffee = (X_test_coffee - mean) / std

y_train_coffee[y_train_coffee == -1] = 0
y_test_coffee[y_test_coffee == -1] = 0

X_train_coffee = X_train_coffee.reshape((-1, 286, 1))
X_test_coffee = X_test_coffee.reshape((-1, 286, 1))

train_dataset_coffee = prepare_dataset(X_train_coffee, y_train_coffee, batch_size=batch_size)
test_dataset_coffee = prepare_dataset(X_test_coffee, y_test_coffee, batch_size=batch_size)

input_shape_3d = (286, 1)
num_classes = len(np.unique(y_train_coffee))
mlp_model = create_mlp_model(input_shape_3d, num_classes)
fcn_model = create_fcn_model(input_shape_3d, num_classes)

In [ ]:
mlp_history, mlp_test_loss, mlp_accuracy = train_model(mlp_model, train_dataset_coffee, test_dataset_coffee, epochs=10, learning_rate=learning_rate)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.6429
Epoch 1: val_loss improved from inf to 0.67908, saving model to best_model.h5
1/1 [==============================] - 1s 1s/step - loss: 0.6931 - accuracy: 0.6429 - val_loss: 0.6791 - val_accuracy: 0.5357
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.6867 - accuracy: 0.5000
Epoch 2: val_loss improved from 0.67908 to 0.67873, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 87ms/step - loss: 0.6867 - accuracy: 0.5000 - val_loss: 0.6787 - val_accuracy: 0.5714
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.7167 - accuracy: 0.5000
Epoch 3: val_loss improved from 0.67873 to 0.67842, saving model to best_model.h5
1/1 [==============================] - 0s 79ms/step - loss: 0.7167 - accuracy: 0.5000 - val_loss: 0.6784 - val_accuracy: 0.5714
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.6874 - accuracy: 0.6071
Epoch 4: val_loss improved from 0.67842 to 0.67817, saving model to best_model.h5
1/1 [==============================] - 0s 78ms/step - loss: 0.6874 - accuracy: 0.6071 - val_loss: 0.6782 - val_accuracy: 0.6071
Epoch 5/10
1/1 [==============================] - ETA: 0s - loss: 0.6637 - accuracy: 0.5357
Epoch 5: val_loss improved from 0.67817 to 0.67783, saving model to best_model.h5
1/1 [==============================] - 0s 83ms/step - loss: 0.6637 - accuracy: 0.5357 - val_lo

In [ ]:
fcn_history, fcn_test_loss, fcn_accuracy = train_model(fcn_model, train_dataset_coffee, test_dataset_coffee, epochs=10, learning_rate=learning_rate)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.6793 - accuracy: 1.0000
Epoch 1: val_loss improved from inf to 0.69507, saving model to best_model.h5
1/1 [==============================] - 3s 3s/step - loss: 0.6793 - accuracy: 1.0000 - val_loss: 0.6951 - val_accuracy: 0.4643
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.6750 - accuracy: 1.0000
Epoch 2: val_loss did not improve from 0.69507
1/1 [==============================] - 1s 695ms/step - loss: 0.6750 - accuracy: 1.0000 - val_loss: 0.6951 - val_accuracy: 0.4643
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.6706 - accuracy: 1.0000
Epoch 3: val_loss did not improve from 0.69507
1/1 [==============================] - 0s 467ms/step - loss: 0.6706 - accuracy: 1.0000 - val_loss: 0.6951 - val_accuracy: 0.4643
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.6663 - accuracy: 1.0000
Epoch 4: val_loss did not improve from 0.69507
1/1 [========================

In [ ]:

fcn_test_loss, fcn_test_accuracy = fcn_model.evaluate(X_test_coffee, y_test_coffee, verbose=0)
print(f"Fully Connected Network Test Accuracy: {fcn_test_accuracy:.4f}")

mlp_test_loss, mlp_test_accuracy = mlp_model.evaluate(X_test_coffee, y_test_coffee, verbose=0)
print(f"MLP Test Accuracy: {mlp_test_accuracy:.4f}")

Fully Connected Network Test Accuracy: 0.4643
MLP Test Accuracy: 0.8929


In [ ]:
X_train_wafer, y_train_wafer = load_UCR_UEA_dataset(name="Wafer", split="train", return_X_y=True, return_type="numpy3D")
X_test_wafer, y_test_wafer = load_UCR_UEA_dataset(name="Wafer", split="test", return_X_y=True, return_type="numpy3D")

if not isinstance(y_train_wafer, np.ndarray):
    y_train_wafer = np.array(y_train_wafer)
if not isinstance(y_test_wafer, np.ndarray):
    y_test_wafer = np.array(y_test_wafer)

y_train_wafer = y_train_wafer.astype('int64')
y_test_wafer = y_test_wafer.astype('int64')

mean = X_train_wafer.mean()
std = X_train_wafer.std()

X_train_wafer = (X_train_wafer - mean) / std
X_test_wafer = (X_test_wafer - mean) / std

y_train_wafer[y_train_wafer == -1] = 0
y_test_wafer[y_test_wafer == -1] = 0

X_train_wafer = X_train_wafer.reshape((-1, 152, 1))
X_test_wafer = X_test_wafer.reshape((-1, 152, 1))

train_dataset_wafer = prepare_dataset(X_train_wafer, y_train_wafer, batch_size=batch_size)
test_dataset_wafer = prepare_dataset(X_test_wafer, y_test_wafer, batch_size=batch_size)

num_classes = len(np.unique(y_train_wafer))

input_shape_3d = (152, 1)
mlp_model = create_mlp_model(input_shape_3d, num_classes)
fcn_model = create_fcn_model(input_shape_3d, num_classes)

In [ ]:
mlp_history, mlp_test_loss, mlp_accuracy = train_model(mlp_model, train_dataset_wafer, test_dataset_wafer, epochs=10, learning_rate=learning_rate)

Epoch 1/10
29/32 [==========================>...] - ETA: 0s - loss: 0.4483 - accuracy: 0.8427
Epoch 1: val_loss improved from inf to 0.34238, saving model to best_model.h5
32/32 [==============================] - 2s 38ms/step - loss: 0.4388 - accuracy: 0.8510 - val_loss: 0.3424 - val_accuracy: 0.8921
Epoch 2/10
11/32 [=========>....................] - ETA: 0s - loss: 0.3431 - accuracy: 0.8949

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/32 [==========================>...] - ETA: 0s - loss: 0.3048 - accuracy: 0.9095
Epoch 2: val_loss improved from 0.34238 to 0.26728, saving model to best_model.h5
32/32 [==============================] - 1s 34ms/step - loss: 0.3040 - accuracy: 0.9070 - val_loss: 0.2673 - val_accuracy: 0.8921
Epoch 3/10
29/32 [==========================>...] - ETA: 0s - loss: 0.2594 - accuracy: 0.8998
Epoch 3: val_loss improved from 0.26728 to 0.23263, saving model to best_model.h5
32/32 [==============================] - 1s 35ms/step - loss: 0.2534 - accuracy: 0.9030 - val_loss: 0.2326 - val_accuracy: 0.8921
Epoch 4/10
32/32 [==============================] - ETA: 0s - loss: 0.2279 - accuracy: 0.9060
Epoch 4: val_loss improved from 0.23263 to 0.20691, saving model to best_model.h5
32/32 [==============================] - 2s 51ms/step - loss: 0.2279 - accuracy: 0.9060 - val_loss: 0.2069 - val_accuracy: 0.8934
Epoch 5/10
31/32 [============================>.] - ETA: 0s - loss: 0.1994 - accuracy: 0.9123

In [ ]:
fcn_history, fcn_test_loss, fcn_accuracy = train_model(fcn_model, train_dataset_wafer, test_dataset_wafer, epochs=10, learning_rate=learning_rate)

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 0.6911 - accuracy: 0.5230
Epoch 1: val_loss improved from inf to 0.62590, saving model to best_model.h5
32/32 [==============================] - 30s 873ms/step - loss: 0.6911 - accuracy: 0.5230 - val_loss: 0.6259 - val_accuracy: 0.8921
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.6723 - accuracy: 0.5930
Epoch 2: val_loss improved from 0.62590 to 0.56539, saving model to best_model.h5
32/32 [==============================] - 18s 562ms/step - loss: 0.6723 - accuracy: 0.5930 - val_loss: 0.5654 - val_accuracy: 0.8921
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.6646 - accuracy: 0.6020
Epoch 3: val_loss improved from 0.56539 to 0.51761, saving model to best_model.h5
32/32 [==============================] - 27s 864ms/step - loss: 0.6646 - accuracy: 0.6020 - val_loss: 0.5176 - val_accuracy: 0.8921
Epoch 4/10
32/32 [==============================] - ETA: 0s - loss: 0.6570 - acc

In [ ]:
fcn_test_loss, fcn_test_accuracy = fcn_model.evaluate(X_test_wafer, y_test_wafer, verbose=0)
print(f"Fully Connected Network Test Accuracy: {fcn_test_accuracy:.4f}")

mlp_test_loss, mlp_test_accuracy = mlp_model.evaluate(X_test_wafer, y_test_wafer, verbose=0)
print(f"MLP Test Accuracy: {mlp_test_accuracy:.4f}")

Fully Connected Network Test Accuracy: 0.8921
MLP Test Accuracy: 0.9481


### Task 3: Time series classification using deep learning 3

Next, you can try to further improve your model by selecting **two** of the following ideas:
- Use Bi-Direction LSTM and CNN networks separately, create two to three layers individually, and concatenate them. This means that until the third (or second) layer, you have two different networks handling the same dataset, and after that, you concatenate the output and finish with any FCN layer. Check [this post](https://stackoverflow.com/questions/59168306/how-to-combine-lstm-and-cnn-in-timeseries-classification) to get inspired.
- Apply any sktime's transformer (not attention transformer) first to the dataset and run any deep learning model you already developed in Tasks 2 and 3. In this case, you need to choose at least two transformers and apply them together.
- Train the model on multiple similar datasets and test it on one specific test set. Check if the model can be improved if it is trained on multiple datasets (at least five datasets). However, for this, you also need to choose the similar datasets based on their classification and motivate your choise in the report (UCR repository has a specific dataset type such as **AUDIO** or **MOTION**). You could try to crop or pad the time series if you would like to match the sizes.

Choose one model you want from the models you have developed in Tasks 1 and 2. Select one idea, try implementing it, and check if you can improve the performance. Note that you do not need to prove that the accuracy scores increase but must explain your trials. Report test scores on three datasets you chose.

In [59]:
def prepare_dataset(X, y, batch_size=32, shuffle_buffer_size=100):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle_buffer_size > 0:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

In [60]:
pip install sktime

In [61]:
from sktime.classification.deep_learning.cnn import CNNClassifier
from sktime.datasets import load_unit_test
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Conv1D, GRU, Flatten, Input,LSTM
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Bidirectional
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, LSTM, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [62]:
from sktime.datasets import load_UCR_UEA_dataset
X_train_ecg200, y_train_ecg200 = load_UCR_UEA_dataset(name="ECG200", split="train", return_X_y=True, return_type="numpy3D")
X_test_ecg200, y_test_ecg200 = load_UCR_UEA_dataset(name="ECG200", split="test", return_X_y=True, return_type="numpy3D")

if not isinstance(y_train_ecg200, np.ndarray):
    y_train_ecg200 = np.array(y_train_ecg200)
if not isinstance(y_test_ecg200, np.ndarray):
    y_test_ecg200 = np.array(y_test_ecg200)

y_train_ecg200 = y_train_ecg200.astype('int64')
y_test_ecg200 = y_test_ecg200.astype('int64')

mean = X_train_ecg200.mean()
std = X_train_ecg200.std()

X_train_ecg200 = (X_train_ecg200 - mean) / std
X_test_ecg200 = (X_test_ecg200 - mean) / std

y_train_ecg200[y_train_ecg200 == -1] = 0
y_test_ecg200[y_test_ecg200 == -1] = 0

X_train_ecg200 = X_train_ecg200.reshape((-1, 96, 1))
X_test_ecg200 = X_test_ecg200.reshape((-1, 96, 1))

batch_size=32
train_dataset_ecg200 = prepare_dataset(X_train_ecg200, y_train_ecg200, batch_size=batch_size)
test_dataset_ecg200 = prepare_dataset(X_test_ecg200, y_test_ecg200, batch_size=batch_size)

num_classes = len(np.unique(y_train_ecg200))
learning_rate=1e-5
input_shape_3d = (96, 1)

In [63]:
def create_model(input_shape):
    inputs = Input(shape=input_shape)

    side1 = Bidirectional(LSTM(100, return_sequences=True))(inputs)
    side2 = Conv1D(200, kernel_size=3, activation='tanh', padding='same')(inputs)

    merged = Add()([side1, side2])
    outputs = Conv1D(200, kernel_size=3, activation='relu', padding='same')(merged)
    outputs = GlobalMaxPooling1D()(outputs)
    outputs = Dense(100, activation='relu')(outputs)
    outputs = Dense(1, activation='sigmoid')(outputs)

    model = Model(inputs, outputs)

    return model

In [64]:
def train_model(model, train_dataset, test_dataset, epochs, learning_rate):
    optimizer = Adam(learning_rate=learning_rate, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, verbose=1),
        ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
    ]

    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=test_dataset,
        callbacks=callbacks
    )

    try:
        model.load_weights('best_model.h5')
    except IOError as e:
        print(f"Checkpoint file not found: {e}. Using model as is after training.")

    test_loss, test_accuracy = model.evaluate(test_dataset)
    return history, test_loss, test_accuracy

In [65]:
input_shape = (96, 1)

model = create_model(input_shape)
history, test_loss, test_accuracy = train_model(
    model=model,
    train_dataset=train_dataset_ecg200,
    test_dataset=test_dataset_ecg200,
    epochs=15,
    learning_rate=1e-4
)

print(f"Test accuracy: {test_accuracy}")

Epoch 1/15
4/4 [==============================] - ETA: 0s - loss: 0.7421 - accuracy: 0.3500
Epoch 1: val_loss improved from inf to 0.66947, saving model to best_model.h5
4/4 [==============================] - 13s 2s/step - loss: 0.7421 - accuracy: 0.3500 - val_loss: 0.6695 - val_accuracy: 0.6400
Epoch 2/15


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - ETA: 0s - loss: 0.6433 - accuracy: 0.6900
Epoch 2: val_loss improved from 0.66947 to 0.64436, saving model to best_model.h5
4/4 [==============================] - 2s 591ms/step - loss: 0.6433 - accuracy: 0.6900 - val_loss: 0.6444 - val_accuracy: 0.6400
Epoch 3/15
4/4 [==============================] - ETA: 0s - loss: 0.6055 - accuracy: 0.6900
Epoch 3: val_loss improved from 0.64436 to 0.63907, saving model to best_model.h5
4/4 [==============================] - 2s 586ms/step - loss: 0.6055 - accuracy: 0.6900 - val_loss: 0.6391 - val_accuracy: 0.6400
Epoch 4/15
4/4 [==============================] - ETA: 0s - loss: 0.5925 - accuracy: 0.6900
Epoch 4: val_loss did not improve from 0.63907
4/4 [==============================] - 2s 574ms/step - loss: 0.5925 - accuracy: 0.6900 - val_loss: 0.6452 - val_accuracy: 0.6400
Epoch 5/15
4/4 [==============================] - ETA: 0s - loss: 0.5862 - accuracy: 0.6900
Epoch 5: val_loss improved from 0.63907 to 0.

In [66]:
print(f"Test Accuracy on ECG dataset: {test_accuracy:.4f}")

Test Accuracy on ECG dataset: 0.6400


In [67]:
X_train_coffee, y_train_coffee = load_UCR_UEA_dataset(name="Coffee", split="train", return_X_y=True, return_type="numpy3D")
X_test_coffee, y_test_coffee = load_UCR_UEA_dataset(name="Coffee", split="test", return_X_y=True, return_type="numpy3D")
X_train_coffee.shape

(28, 1, 286)

In [68]:
#coffee dataset
X_train_coffee, y_train_coffee = load_UCR_UEA_dataset(name="Coffee", split="train", return_X_y=True, return_type="numpy3D")
X_test_coffee, y_test_coffee = load_UCR_UEA_dataset(name="Coffee", split="test", return_X_y=True, return_type="numpy3D")

if not isinstance(y_train_coffee, np.ndarray):
    y_train_coffee = np.array(y_train_coffee)
if not isinstance(y_test_coffee, np.ndarray):
    y_test_coffee = np.array(y_test_coffee)

y_train_coffee = y_train_coffee.astype('int64')
y_test_coffee = y_test_coffee.astype('int64')

mean = X_train_coffee.mean()
std = X_train_coffee.std()

X_train_coffee = (X_train_coffee - mean) / std
X_test_coffee = (X_test_coffee - mean) / std

y_train_coffee[y_train_coffee == -1] = 0
y_test_coffee[y_test_coffee == -1] = 0

X_train_coffee = X_train_coffee.reshape((-1, 286, 1))
X_test_coffee = X_test_coffee.reshape((-1, 286, 1))

train_dataset_coffee = prepare_dataset(X_train_coffee, y_train_coffee, batch_size=batch_size)
test_dataset_coffee = prepare_dataset(X_test_coffee, y_test_coffee, batch_size=batch_size)

input_shape_3d = (286, 1)
num_classes = len(np.unique(y_train_coffee))

In [69]:
model = create_model(input_shape_3d)
history, test_loss, test_accuracy = train_model(
    model=model,
    train_dataset=train_dataset_coffee,
    test_dataset=test_dataset_coffee,
    epochs=15,
    learning_rate=1e-4
)

print(f"Test accuracy: {test_accuracy}")

Epoch 1/15
1/1 [==============================] - ETA: 0s - loss: 0.6977 - accuracy: 0.3571
Epoch 1: val_loss improved from inf to 0.69474, saving model to best_model.h5
1/1 [==============================] - 16s 16s/step - loss: 0.6977 - accuracy: 0.3571 - val_loss: 0.6947 - val_accuracy: 0.3929
Epoch 2/15


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - ETA: 0s - loss: 0.6944 - accuracy: 0.3929
Epoch 2: val_loss improved from 0.69474 to 0.69241, saving model to best_model.h5
1/1 [==============================] - 2s 2s/step - loss: 0.6944 - accuracy: 0.3929 - val_loss: 0.6924 - val_accuracy: 0.5714
Epoch 3/15
1/1 [==============================] - ETA: 0s - loss: 0.6917 - accuracy: 0.6071
Epoch 3: val_loss improved from 0.69241 to 0.68979, saving model to best_model.h5
1/1 [==============================] - 3s 3s/step - loss: 0.6917 - accuracy: 0.6071 - val_loss: 0.6898 - val_accuracy: 0.9286
Epoch 4/15
1/1 [==============================] - ETA: 0s - loss: 0.6892 - accuracy: 0.8929
Epoch 4: val_loss improved from 0.68979 to 0.68771, saving model to best_model.h5
1/1 [==============================] - 2s 2s/step - loss: 0.6892 - accuracy: 0.8929 - val_loss: 0.6877 - val_accuracy: 0.6071
Epoch 5/15
1/1 [==============================] - ETA: 0s - loss: 0.6873 - accuracy: 0.6071
Epoch 5: val_loss i

In [70]:
print(f"Test Accuracy on coffee dataset: {test_accuracy:.4f}")

Test Accuracy on coffee dataset: 0.9643


In [71]:
X_train_wafer, y_train_wafer = load_UCR_UEA_dataset(name="Wafer", split="train", return_X_y=True, return_type="numpy3D")
X_test_wafer, y_test_wafer = load_UCR_UEA_dataset(name="Wafer", split="test", return_X_y=True, return_type="numpy3D")

if not isinstance(y_train_wafer, np.ndarray):
    y_train_wafer = np.array(y_train_wafer)
if not isinstance(y_test_wafer, np.ndarray):
    y_test_wafer = np.array(y_test_wafer)

y_train_wafer = y_train_wafer.astype('int64')
y_test_wafer = y_test_wafer.astype('int64')

mean = X_train_wafer.mean()
std = X_train_wafer.std()

X_train_wafer = (X_train_wafer - mean) / std
X_test_wafer = (X_test_wafer - mean) / std

y_train_wafer[y_train_wafer == -1] = 0
y_test_wafer[y_test_wafer == -1] = 0

X_train_wafer = X_train_wafer.reshape((-1, 152, 1))
X_test_wafer = X_test_wafer.reshape((-1, 152, 1))

train_dataset_wafer = prepare_dataset(X_train_wafer, y_train_wafer, batch_size=batch_size)
test_dataset_wafer = prepare_dataset(X_test_wafer, y_test_wafer, batch_size=batch_size)

num_classes = len(np.unique(y_train_wafer))

input_shape_3d = (152, 1)

In [72]:
model = create_model(input_shape_3d)

history, test_loss, test_accuracy = train_model(
    model=model,
    train_dataset=train_dataset_wafer,
    test_dataset=test_dataset_wafer,
    epochs=10,
    learning_rate=1e-4
)

print(f"Test accuracy: {test_accuracy}")

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 0.3869 - accuracy: 0.9030
Epoch 1: val_loss improved from inf to 0.34338, saving model to best_model.h5
32/32 [==============================] - 52s 1s/step - loss: 0.3869 - accuracy: 0.9030 - val_loss: 0.3434 - val_accuracy: 0.8921
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


32/32 [==============================] - ETA: 0s - loss: 0.3205 - accuracy: 0.9030
Epoch 2: val_loss improved from 0.34338 to 0.33073, saving model to best_model.h5
32/32 [==============================] - 38s 1s/step - loss: 0.3205 - accuracy: 0.9030 - val_loss: 0.3307 - val_accuracy: 0.8921
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.3078 - accuracy: 0.9030
Epoch 3: val_loss improved from 0.33073 to 0.32118, saving model to best_model.h5
32/32 [==============================] - 33s 1s/step - loss: 0.3078 - accuracy: 0.9030 - val_loss: 0.3212 - val_accuracy: 0.8921
Epoch 4/10
32/32 [==============================] - ETA: 0s - loss: 0.2983 - accuracy: 0.9030
Epoch 4: val_loss improved from 0.32118 to 0.31198, saving model to best_model.h5
32/32 [==============================] - 30s 946ms/step - loss: 0.2983 - accuracy: 0.9030 - val_loss: 0.3120 - val_accuracy: 0.8921
Epoch 5/10
32/32 [==============================] - ETA: 0s - loss: 0.2878 - accuracy: 0.9030

In [73]:
print(f"Test Accuracy on wafer dataset: {test_accuracy:.4f}")

Test Accuracy on wafer dataset: 0.9711


In [ ]:
!pip install sktime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 13.2 MB/s eta 0:00:00


In [ ]:
#Transformation using BoxCoxTransformer and Rocket
from sktime.datasets import load_UCR_UEA_dataset
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.transformations.panel.rocket import Rocket
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
from tensorflow.keras.optimizers import Adam

X_train, y_train = load_UCR_UEA_dataset(name="ECG200", split="train", return_X_y=True, return_type="numpy3D")
X_test, y_test = load_UCR_UEA_dataset(name="ECG200", split="test", return_X_y=True, return_type="numpy3D")

y_train = y_train.astype('int64')
y_test = y_test.astype('int64')

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

boxcox_trans = BoxCoxTransformer()
rocket_trans = Rocket()

X_train_boxcox = boxcox_trans.fit_transform(X_train)
X_test_boxcox = boxcox_trans.transform(X_test)

rocket_trans.fit(X_train_boxcox)
X_train_rocket = rocket_trans.transform(X_train_boxcox)
X_test_rocket = rocket_trans.transform(X_test_boxcox)

num_classes = len(np.unique(y_train))
num_features = X_train_rocket.shape[1]

In [ ]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [ ]:
def create_mlp_model(input_shape, num_classes):
    output_activation = 'sigmoid' if num_classes == 2 else 'softmax'

    output_neurons = 1 if num_classes == 2 else num_classes

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(output_neurons, activation=output_activation)
    ])

    chosen_loss = 'binary_crossentropy' if num_classes == 2 else 'categorical_crossentropy'

    optimizer = Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=chosen_loss, metrics=['accuracy'])

    return model


In [ ]:
print("X_train_rocket shape:", X_train_rocket.shape)
print("y_train shape:", y_train.shape)
print("X_test_rocket shape:", X_test_rocket.shape)
print("y_test shape:", y_test.shape)


X_train_rocket shape: (100, 20000)
y_train shape: (100,)
X_test_rocket shape: (100, 20000)
y_test shape: (100,)


In [ ]:
import tensorflow as tf

input_shape = X_train_rocket.shape[1:]

mlp_model = create_mlp_model(input_shape, num_classes)

history = mlp_model.fit(
X_train_rocket, y_train,
validation_data=(X_test_rocket, y_test),
epochs=10,
batch_size=32
)

Epoch 1/10
4/4 [==============================] - 2s 175ms/step - loss: 6.4375 - accuracy: 0.6300 - val_loss: 0.7370 - val_accuracy: 0.5300
Epoch 2/10
4/4 [==============================] - 0s 79ms/step - loss: 3.0892 - accuracy: 0.5000 - val_loss: 4.4113 - val_accuracy: 0.6400
Epoch 3/10
4/4 [==============================] - 0s 83ms/step - loss: 3.7889 - accuracy: 0.6800 - val_loss: 1.3375 - val_accuracy: 0.6400
Epoch 4/10
4/4 [==============================] - 0s 80ms/step - loss: 2.1252 - accuracy: 0.5700 - val_loss: 0.6028 - val_accuracy: 0.7300
Epoch 5/10
4/4 [==============================] - 0s 77ms/step - loss: 1.9776 - accuracy: 0.6300 - val_loss: 0.6128 - val_accuracy: 0.7300
Epoch 6/10
4/4 [==============================] - 0s 87ms/step - loss: 1.6946 - accuracy: 0.4800 - val_loss: 0.7803 - val_accuracy: 0.7200
Epoch 7/10
4/4 [==============================] - 0s 88ms/step - loss: 1.4273 - accuracy: 0.6700 - val_loss: 0.8360 - val_accuracy: 0.7000
Epoch 8/10
4/4 [==========

In [ ]:
test_loss, test_accuracy = mlp_model.evaluate(X_test_rocket, y_test, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Loss: 1.1429
Test Accuracy: 0.6500


In [ ]:
#Same on coffee dataset
from sktime.datasets import load_UCR_UEA_dataset
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.transformations.panel.rocket import Rocket
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
from tensorflow.keras.optimizers import Adam

X_train, y_train = load_UCR_UEA_dataset(name="Coffee", split="train", return_X_y=True, return_type="numpy3D")
X_test, y_test = load_UCR_UEA_dataset(name="Coffee", split="test", return_X_y=True, return_type="numpy3D")

y_train = y_train.astype('int64')
y_test = y_test.astype('int64')

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

boxcox_trans = BoxCoxTransformer()
rocket_trans = Rocket()

X_train_boxcox = boxcox_trans.fit_transform(X_train)
X_test_boxcox = boxcox_trans.transform(X_test)

rocket_trans.fit(X_train_boxcox)
X_train_rocket = rocket_trans.transform(X_train_boxcox)
X_test_rocket = rocket_trans.transform(X_test_boxcox)

num_classes = len(np.unique(y_train))
num_features = X_train_rocket.shape[1]

In [ ]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [ ]:
def create_mlp_model(input_shape, num_classes):
    output_activation = 'sigmoid' if num_classes == 2 else 'softmax'

    output_neurons = 1 if num_classes == 2 else num_classes

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(output_neurons, activation=output_activation)
    ])

    chosen_loss = 'binary_crossentropy' if num_classes == 2 else 'categorical_crossentropy'

    optimizer = Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=chosen_loss, metrics=['accuracy'])

    return model

In [ ]:
import tensorflow as tf

input_shape = X_train_rocket.shape[1:]
mlp_model = create_mlp_model(input_shape, num_classes)

history = mlp_model.fit(
X_train_rocket, y_train,
validation_data=(X_test_rocket, y_test),
epochs=10,
batch_size=32
)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 5.2224 - accuracy: 0.5000 - val_loss: 18.5939 - val_accuracy: 0.4643
Epoch 2/10
1/1 [==============================] - 0s 115ms/step - loss: 17.6225 - accuracy: 0.5000 - val_loss: 14.5969 - val_accuracy: 0.4643
Epoch 3/10
1/1 [==============================] - 0s 120ms/step - loss: 13.3944 - accuracy: 0.5000 - val_loss: 7.6235 - val_accuracy: 0.4643
Epoch 4/10
1/1 [==============================] - 0s 111ms/step - loss: 5.8456 - accuracy: 0.5000 - val_loss: 0.7723 - val_accuracy: 0.4643
Epoch 5/10
1/1 [==============================] - 0s 117ms/step - loss: 2.4441 - accuracy: 0.5714 - val_loss: 3.1969 - val_accuracy: 0.5357
Epoch 6/10
1/1 [==============================] - 0s 118ms/step - loss: 5.3080 - accuracy: 0.3929 - val_loss: 3.3062 - val_accuracy: 0.5357
Epoch 7/10
1/1 [==============================] - 0s 117ms/step - loss: 5.5706 - accuracy: 0.4643 - val_loss: 1.6731 - val_accuracy: 0.5357
Epoch 8/10
1/1 [===

In [ ]:
test_loss, test_accuracy = mlp_model.evaluate(X_test_rocket, y_test, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Loss: 3.1160
Test Accuracy: 0.4643


In [ ]:
from sktime.datasets import load_UCR_UEA_dataset
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.transformations.panel.rocket import Rocket
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
from tensorflow.keras.optimizers import Adam

X_train, y_train = load_UCR_UEA_dataset(name="BirdChicken", split="train", return_X_y=True, return_type="numpy3D")
X_test, y_test = load_UCR_UEA_dataset(name="BirdChicken", split="test", return_X_y=True, return_type="numpy3D")
y_train = y_train.astype('int64')
y_test = y_test.astype('int64')

y_train[y_train == 2] = 0
y_test[y_test == 2] = 0

boxcox_trans = BoxCoxTransformer()
rocket_trans = Rocket()

X_train_boxcox = boxcox_trans.fit_transform(X_train)
X_test_boxcox = boxcox_trans.transform(X_test)

rocket_trans.fit(X_train_boxcox)
X_train_rocket = rocket_trans.transform(X_train_boxcox)
X_test_rocket = rocket_trans.transform(X_test_boxcox)

num_classes = len(np.unique(y_train))
num_features = X_train_rocket.shape[1]

In [ ]:
def create_mlp_model(input_shape, num_classes):
    output_activation = 'sigmoid' if num_classes == 2 else 'softmax'

    output_neurons = 1 if num_classes == 2 else num_classes

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(output_neurons, activation=output_activation)
    ])

    chosen_loss = 'binary_crossentropy' if num_classes == 2 else 'categorical_crossentropy'

    optimizer = Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=chosen_loss, metrics=['accuracy'])

    return model

In [ ]:
import tensorflow as tf

input_shape = X_train_rocket.shape[1:]
mlp_model = create_mlp_model(input_shape, num_classes)

history = mlp_model.fit(
X_train_rocket, y_train,
validation_data=(X_test_rocket, y_test),
epochs=10,
batch_size=32
)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 3.5566 - accuracy: 0.4500 - val_loss: 12.1570 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 134ms/step - loss: 12.2112 - accuracy: 0.5000 - val_loss: 5.8757 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 105ms/step - loss: 4.4869 - accuracy: 0.5500 - val_loss: 0.8871 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 97ms/step - loss: 1.3882 - accuracy: 0.6000 - val_loss: 3.4902 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 110ms/step - loss: 4.0473 - accuracy: 0.3500 - val_loss: 2.9224 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 103ms/step - loss: 3.8088 - accuracy: 0.4500 - val_loss: 1.0655 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 129ms/step - loss: 2.2386 - accuracy: 0.5500 - val_loss: 0.7328 - val_accuracy: 0.4500
Epoch 8/10
1/1 [======

In [ ]:
test_loss, test_accuracy = mlp_model.evaluate(X_test_rocket, y_test, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Loss: 1.8238
Test Accuracy: 0.5000


### Task 4: Time series classification using sktime

We can use RandomizedSearch to find optimal parameter options on pipelines. However, sktime's pipeline does not support scikit-learn's classifiers such as DecisionTree or RandomForest well. However, sometimes we would like to use the output of the sktime transformer (e.g., catch22) to train scikit-learn models such as RandomForest. Sktime supports this with `SklearnClassifierPipeline` to put a  scikit-learn classifier and sktime's transformer together and you need to implement it.

Pick one classifier from scikit-learn (that can be anything! e.g., Decision Tree or Logistic Regressor) and two transformers from sktime and create `SklearnClassifierPipeline.` As we tried in this lab, that can be **Rocket with RandomForest** or **Catch22 with DecisionTree**. Pick one parameter from each module (in total, three, one from the classifier and two from two transformers) and run a randomized search on the pipeline you define and report the test score of the best model found by the randomized Search. Compare your best score to the score from the same model with the default setting.

Task 4 involves a time-consuming process, so you can only choose **one dataset** to perform the task above. Also, note that you do not need to perform better by conducting a randomized search for this task (but still good to try!).

In [ ]:
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.catch22 import Catch22
from sktime.pipeline import make_pipeline
from sktime.classification.compose import SklearnClassifierPipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sktime.datasets import load_UCR_UEA_dataset
from sklearn.metrics import accuracy_score
from sktime.transformations.series.exponent import ExponentTransformer

X_train, y_train = load_UCR_UEA_dataset(name="Coffee", split="train", return_X_y=True)
X_test, y_test = load_UCR_UEA_dataset(name="Coffee", split="test", return_X_y=True)


In [ ]:
from sktime.pipeline import make_pipeline

rocket_transformer = Rocket()
rf_classifier = RandomForestClassifier()
square = ExponentTransformer()

pipeline = make_pipeline(rocket_transformer,square , rf_classifier)

param_grid = {
    'rocketclassifier__num_kernels': [1, 10, 20, 100, 1000],
    'exponenttransformer__power': [1,2,3,4,5],
    'randomforestclassifier__n_estimators': [100, 200, 300]
}

In [ ]:
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=3,
    scoring='accuracy',
    cv=5,
    random_state=42)

random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

test_score = accuracy_score(y_test, y_pred)
default_model = SklearnClassifierPipeline(
    RandomForestClassifier(),[
    ('transformer1', Rocket()),
    ('transformer2', ExponentTransformer())]
)
default_model.fit(X_train, y_train)
default_pred = default_model.predict(X_test)
default_score = accuracy_score(y_test, default_pred)

In [ ]:
print(f"Test score of the best model: {test_score}")
print(f"Test score with default settings: {default_score}")


Test score of the best model: 1.0
Test score with default settings: 1.0


### Task 5: Multivariate time series classification

Time series can be **multivariate**, which means there can be many values (= data points) describing one time point. In this task, you will use one **multivariate** dataset (**Eplipsy**) and try to run one deep learning model and one sktime model to see if those models work well on multivariate time series.

- Use Eplipsy dataset in the UCR/UEA repository.
- Run two classifiers of your choice in sktime, such as TapNet, Rocket, or MiniRocket, together with **the best tensorflow deep learning model from the previous tasks** on Eplipsy. You need to adjust the deep learning model's input layer to handle this multivariate dataset.
- Use sktime's `load_UCR_UEA_dataset` function to perform. You should use each dataset's original train/test splits.
- For the deep learning model, you should transform it using TensorFlow data API (`tf.data`) to manage your dataset and use `shuffle`, `batch`, and `prefetch` functions. This means that you need to create the validation set first. If you use Torch, explain how you implement the equivalent operations.
- For training, you need to run at least 10 epochs for your deep learning model. For TapNet, you can keep the default parameter options.
- Report the test scores of three models on the predefined test set.
- **Do the same task on one more chosen multivariate time series dataset.**

In [ ]:
from sktime.datasets import load_UCR_UEA_dataset
X_train_epilepsy, y_train_epilepsy = load_UCR_UEA_dataset(name="Epilepsy", split="train", return_X_y=True, return_type="numpy3D")
X_test_epilepsy, y_test_epilepsy = load_UCR_UEA_dataset(name="Epilepsy", split="test", return_X_y=True, return_type="numpy3D")

In [ ]:
def prepare_dataset(X, y, batch_size=32, shuffle_buffer_size=100):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle_buffer_size > 0:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

batch_size = 32

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Conv1D, GRU, Flatten, Input
from tensorflow.keras.layers import MaxPooling1D

if not isinstance(y_train_epilepsy, np.ndarray):
    y_train_epilepsy = np.array(y_train_epilepsy)
if not isinstance(y_test_epilepsy, np.ndarray):
    y_test_epilepsy = np.array(y_test_epilepsy)

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train_epilepsy)
y_test_encoded = label_encoder.transform(y_test_epilepsy)

y_train_epilepsy = y_train_encoded.astype('int64')
y_test_epilepsy = y_test_encoded.astype('int64')

def prepare_dataset(X, y, batch_size=32, shuffle_buffer_size=100):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle_buffer_size > 0:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

batch_size = 32

In [ ]:
def create_fcn_model_multivariate(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
train_dataset = prepare_dataset(X_train_epilepsy,y_train_epilepsy)
test_dataset = prepare_dataset(X_test_epilepsy, y_test_epilepsy)

In [ ]:
input_shape = X_train_epilepsy.shape[1:]
num_classes = len(np.unique(y_train_epilepsy))

In [ ]:
model = create_fcn_model_multivariate(input_shape, num_classes)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
5/5 [==============================] - 1s 63ms/step - loss: 1.2994 - accuracy: 0.4307 - val_loss: 1.0049 - val_accuracy: 0.5652
Epoch 2/10
5/5 [==============================] - 0s 30ms/step - loss: 0.6932 - accuracy: 0.7664 - val_loss: 0.7348 - val_accuracy: 0.7029
Epoch 3/10
5/5 [==============================] - 0s 17ms/step - loss: 0.3504 - accuracy: 0.8832 - val_loss: 0.6405 - val_accuracy: 0.7464
Epoch 4/10
5/5 [==============================] - 0s 28ms/step - loss: 0.1716 - accuracy: 0.9708 - val_loss: 0.9505 - val_accuracy: 0.6739
Epoch 5/10
5/5 [==============================] - 0s 16ms/step - loss: 0.0692 - accuracy: 0.9781 - val_loss: 1.0066 - val_accuracy: 0.6739
Epoch 6/10
5/5 [==============================] - 0s 17ms/step - loss: 0.0266 - accuracy: 1.0000 - val_loss: 0.7376 - val_accuracy: 0.7319
Epoch 7/10
5/5 [==============================] - 0s 16ms/step - loss: 0.0155 - accuracy: 1.0000 - val_loss: 1.2482 - val_accuracy: 0.6812
Epoch 8/10
5/5 [===========

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test accuracy: {test_accuracy:.4f}")


5/5 [==============================] - 0s 9ms/step - loss: 0.9297 - accuracy: 0.7464
Test accuracy: 0.7464


In [ ]:
pip install keras-self-attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18894 sha256=902e1a2fda477e2853c1b31c27ab932967e6cc25e53f76946229754681c32a27
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras-self-attention


In [ ]:
from sktime.classification.deep_learning import TapNetClassifier

tapnet = TapNetClassifier(n_epochs=10)
tapnet.fit(X_train_epilepsy, y_train_epilepsy)
tapnet_score = tapnet.score(X_test_epilepsy, y_test_epilepsy)

9/9 [==============================] - 3s 279ms/step


In [ ]:
print(f"Accuracy of TapNet classifier: {tapnet_score:.2f}")

Accuracy of TapNet classifier: 0.92


In [ ]:
from sktime.classification.kernel_based import RocketClassifier
rocket = RocketClassifier(n_jobs = -1)
rocket.fit(X_train_epilepsy, y_train_epilepsy)
rocket_score =rocket.score(X_test_epilepsy, y_test_epilepsy)

In [ ]:
print(f"Accuracy of Rocket classifier: {rocket_score:.2f}")

Accuracy of Rocket classifier: 0.99


In [ ]:
#Cricket dataset
from sktime.datasets import load_UCR_UEA_dataset
X_train_cr, y_train_cr = load_UCR_UEA_dataset(name="Cricket", split="train", return_X_y=True, return_type="numpy3D")
X_test_cr, y_test_cr = load_UCR_UEA_dataset(name="Cricket", split="test", return_X_y=True, return_type="numpy3D")

In [ ]:
pip install sktime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 8.1 MB/s eta 0:00:00


In [ ]:

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Conv1D, GRU, Flatten, Input
from tensorflow.keras.layers import MaxPooling1D

if not isinstance(y_train_cr, np.ndarray):
    y_train_cr = np.array(y_train_cr)
if not isinstance(y_test_cr, np.ndarray):
    y_test_cr = np.array(y_test_cr)

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train_cr)
y_test_encoded = label_encoder.transform(y_test_cr)

y_train_cr = y_train_encoded.astype('int64')
y_test_cr = y_test_encoded.astype('int64')

def prepare_dataset(X, y, batch_size=32, shuffle_buffer_size=100):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle_buffer_size > 0:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

batch_size = 32

In [ ]:
mean = X_train_cr.mean()
std = X_train_cr.std()

X_train_normalized = (X_train_cr - mean) / std
X_test_normalized = (X_test_cr - mean) / std

In [ ]:
#instances,dimension,timestep
X_train_cr.shape

(108, 6, 1197)

In [ ]:
X_train_cr= X_train_cr.reshape((-1, 1197, 6))
X_test_cr = X_test_cr.reshape((-1, 1197, 6))

In [ ]:
train_dataset = prepare_dataset(X_train_cr,y_train_cr)
test_dataset = prepare_dataset(X_test_cr, y_test_cr)

In [ ]:
input_shape = X_train_cr.shape[1:]
num_classes = len(np.unique(y_train_cr))

In [ ]:
model = create_fcn_model_multivariate(input_shape, num_classes)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
4/4 [==============================] - 2s 123ms/step - loss: 2.2322 - accuracy: 0.1944 - val_loss: 1.3184 - val_accuracy: 0.5556
Epoch 2/10
4/4 [==============================] - 0s 66ms/step - loss: 0.7910 - accuracy: 0.7963 - val_loss: 0.8146 - val_accuracy: 0.7500
Epoch 3/10
4/4 [==============================] - 0s 55ms/step - loss: 0.1837 - accuracy: 0.9722 - val_loss: 0.6676 - val_accuracy: 0.8333
Epoch 4/10
4/4 [==============================] - 0s 51ms/step - loss: 0.0507 - accuracy: 0.9907 - val_loss: 0.5704 - val_accuracy: 0.8472
Epoch 5/10
4/4 [==============================] - 0s 51ms/step - loss: 0.0189 - accuracy: 0.9907 - val_loss: 0.7626 - val_accuracy: 0.9028
Epoch 6/10
4/4 [==============================] - 0s 53ms/step - loss: 0.0265 - accuracy: 0.9907 - val_loss: 0.6981 - val_accuracy: 0.8750
Epoch 7/10
4/4 [==============================] - 0s 51ms/step - loss: 0.1657 - accuracy: 0.9722 - val_loss: 0.9597 - val_accuracy: 0.8611
Epoch 8/10
4/4 [==========

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test accuracy: {test_accuracy:.4f}")

3/3 [==============================] - 0s 16ms/step - loss: 0.9472 - accuracy: 0.8611
Test accuracy: 0.8611


In [ ]:
from sktime.classification.deep_learning import TapNetClassifier

tapnet = TapNetClassifier(n_epochs=10)
tapnet.fit(X_train_cr, y_train_cr)
tapnet_score = tapnet.score(X_test_cr, y_test_cr)

5/5 [==============================] - 2s 16ms/step


In [ ]:
print(f"Accuracy of TapNet classifier: {tapnet_score:.2f}")

Accuracy of TapNet classifier: 0.40


In [ ]:
from sktime.classification.kernel_based import RocketClassifier
rocket = RocketClassifier(n_jobs = -1)
rocket.fit(X_train_cr, y_train_cr)
rocket_score =rocket.score(X_test_cr, y_test_cr)

In [ ]:
print(f"Accuracy of Rocket classifier: {rocket_score:.2f}")

Accuracy of Rocket classifier: 0.38


### Put everything together

You have tested various models on your four chosen datasets. Which model shows the best performance? You need to write a simple report which should be max two pages about your trials.
 - Task 0: State three univariate time series datasets you chose for the tasks.
 - Task 1: Report the average test scores of four models on three datasets you chose. It would be four scores in total. Mark the best model in terms of the average test score. Briefly explain the structure you constructed.
 - Task 2: Report test scores of three models on three datasets you chose. It would be nine scores in total. Report the rank of average accuracy scores of three models. Briefly explain two deep learning model structures you constructed to confirm that you correctly developed the models in the paper.
 - Task 3: Briefly explain which model you chose, how you improved it. Report test scores of the model on three datasets you chose.
 - Task 4: Explain your choice of dataset and (classifier, transformer) pair and the parameters you tried to optimize. Report the best score and estimator from the randomized search instance and the test score of the best model.  Compare your best score to the score from the same model with the default setting.
 - Task 5: Report the best model (your two chosen classifiers vs your deep learning model) in terms of the test score. Briefly explain how you handle the multivariate dataset for your two models.